In [1]:
from Adafruit_IO import Client, Feed
from i2c_sensor_classes import Ky015Sensor, Ky018Sensor
from aio_helper_classes import HabitatMonitorClient
import time
import threading
import asyncio

In [2]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
base = BaseOverlay("base.bit")

# Testing Reading from the Sensors 
no cloud here, trying to decouple cloud code and sensor reading code

In [3]:
KY015_ADDR = 8
KY018_ADDR = 0x28
SCL_PIN = 2
SDA_PIN = 3

liba = MicroblazeLibrary(base.PMODA, ['i2c'])
#custom class see 'src/i2c_sensor_classes.py'
ky015 = Ky015Sensor(liba, SDA_PIN, SCL_PIN, KY015_ADDR)

libb = MicroblazeLibrary(base.PMODB, ['i2c'])
#custom class see 'src/i2c_sensor_classes.py'
ky018 = Ky018Sensor(libb, SDA_PIN, SCL_PIN, KY018_ADDR)

response = input('Read Data?(y/n): ')
while response != 'n':
    temp, hum = ky015.get_temp_and_hum_data()
    bright = ky018.get_brightness_data()
    print('\tTemp: {}C, Hum: {}%\n\tBright: {}'.format(temp,hum, bright))
    response = input('Read Data?(y/n): ')

Read Data?(y/n): y
	Temp: 12.0C, Hum: 42.0%
	Bright: 1180
Read Data?(y/n): y
	Temp: 12.0C, Hum: 42.0%
	Bright: 1178
Read Data?(y/n): n


# Testing the Habitat Monitor class
this class extends the default Adafruit_IO Client, with functionality specific to out Habitat Monitor. If check the client_id and either sets the feeds to the correct Feed Objects, or if the client_id is new then new data feeds are created and associated with the passed client-id. 

In [ ]:
with open('adafruitkey.dontpush') as file:
    UNAME = file.readline().rstrip('\n')
    KEY = file.readline().rstrip('\n')
client_id = 'vinit-pynq-board'

#custom class see "src/aio_helper_classes.py"
aio = HabitatMonitorClient(UNAME, KEY, client_id)    

In [ ]:
response = 'y'
while response != 'n':
    response = input('Read Data?(y/n): ')
    if response == 'n': break
    temp, hum = ky015.get_temp_and_hum_data()
    bright = ky018.get_brightness_data()
    print('\tTemp: {}C, Hum: {}%\n\tBright: {}'.format(temp,hum, bright))
    response = input('Publish to Cloud?(y/n): ')
    if response == 'n': break
    aio.send(aio.temperature_feed.key, temp)
    aio.send(aio.humidity_feed.key, hum)
    aio.send(aio.brightness_feed.key, bright)
    print('\tData Sent to Adafrui.io')

# Client Code
## ToDo
- Integrate Threading + LED + Button code from Ryan
- Need to reace to thresholds and blink leds
- Should be able to receive new threshold values from cloud
- should Read AIO uname + key from a local file (dont push the file to git)
- launch all client threads in the client_process fucniton 

## Threads for Client

In [3]:
def client_button(btns):
    global stop
    while not stop:
        if btns.read() == 0x01:
            stop = True
    print("Button was pressed! Ending program...")
    
def client_LED(LEDStates, LEDLock):
    #Process to handle blinking the LEDs to show various statuses on the client board
    blinkrate = 1
    t_on = 1/(2*blinkrate)
    
    blinkOn = False; #want lights to blink in sync, this stores state
    
    while not stop:
        usingLock = LEDLock.acquire(True)
        if usingLock:
            for i in range(4):
                if LEDStates[i] == 1:
                    if blinkOn:
                        base.leds[i].off()
                    else:
                        base.leds[i].on()

            for i in range(4,6):
                if LEDStates[i] == 1:
                    if blinkOn:
                        base.rgbleds[i].off()
                    else:
                        base.rgbleds[i].on(0b100)
            LEDLock.release()
        if blinkOn:
            blinkOn = False
        else:
            blinkOn = True
                 
        time.sleep(t_on)
    
def start_blink(LEDStates,index):
    LEDStates[index] = 1
    
def stop_blink(LEDStates,index):
    LEDStates[index] = 0
    led_off(LEDStates,index)
    
def blinkAllLEDs():
    for i in LEDStates:
        i = 1
    
def led_off(LEDStates,index):
    if index < 0 or index > 6:
        return
    elif(LEDStates[index] == 1):
        return
    elif index < 4:
        base.leds[index].off()
    else:
        base.rgbleds[index].off()
        
def led_on(LEDStates,index):
    if index < 0 or index > 6:
        return
    elif(LEDStates[index] == 1):
        return
    elif index < 4:
        base.leds[index].on()
    else:
        base.rgbleds[index].on(0b100)

In [8]:
def client_sensors(aio,LEDStates,LEDLock):
    asyncio.set_event_loop(asyncio.new_event_loop()) #required to avoid no event loop error
    
    KY015_ADDR = 8
    KY018_ADDR = 0x28
    SCL_PIN = 2
    SDA_PIN = 3

    liba = MicroblazeLibrary(base.PMODA, ['i2c'])
    #custom class see 'src/i2c_sensor_classes.py'
    ky015 = Ky015Sensor(liba, SDA_PIN, SCL_PIN, KY015_ADDR)

    libb = MicroblazeLibrary(base.PMODB, ['i2c'])
    #custom class see 'src/i2c_sensor_classes.py'
    ky018 = Ky018Sensor(libb, SDA_PIN, SCL_PIN, KY018_ADDR)

    lbtFeed = aio.feeds('low-brightness-threshold')
    hbtFeed = aio.feeds('high-brightness-threshold')
    lttFeed = aio.feeds('low-temperature-threshold')
    httFeed = aio.feeds('high-temperature-threshold')
    lhtFeed = aio.feeds('low-humidity-threshold')
    hhtFeed = aio.feeds('high-humidity-threshold')
    
    tempIsGood = True
    thresholdsUpToDate = True
    samplectr = 0
    numberOfFailedPushes = 0
    while not stop:
        start_time = time.time()
        if samplectr == 0:
            led_on(LEDStates,2) #solid LED 2 indicates updating thresholds
            try:
                [lowB, highB] = updateBrightnessThresholds(aio,lbtFeed,hbtFeed)
                [lowT, highT] = updateTemperatureThresholds(aio,lttFeed,httFeed)
                [lowH, highH] = updateHumidityThresholds(aio,lhtFeed,hhtFeed)
                if not thresholdsUpToDate:
                    thresholdsUpToDate = True
                    usinglock = LEDLock.acquire(True)
                    if usinglock:
                        stop_blink(LEDStates,2)
                        LEDLock.release()
            except:
                thresholdsUpToDate = False
                print("Error updating thresholds")
                start_blink(LEDStates,2) #blinking LED 2 indicates threshold may be out of date
            led_off(LEDStates,2)
        samplectr = (samplectr + 1) % SAMPLES_BEFORE_THRESHOLD_UPDATE
        
        temp, hum = ky015.get_temp_and_hum_data()
        bright = ky018.get_brightness_data()
        print('\tTemp: {}C, Hum: {}%\n\tBright: {}'.format(temp,hum, bright))
        
        #Temperature check
        if temp < lowT:
            tempIsGood = False
            start_blink(LEDStates,4)
            print("ALERT: Temperature is lower than threshold!")
        elif temp > highT:
            tempIsGood = False
            start_blink(LEDStates,4)
            print("ALERT: Temperature is higher than threshold!")
        else:
            tempIsGood = True
        
        #Humidity check
        if hum < lowH:
            start_blink(LEDStates,4)
            print("ALERT: Humidity is lower than threshold!")
        elif hum > highH:
            start_blink(LEDStates,4)
            print("ALERT: Humidity is higher than threshold!")
        else:
            if tempIsGood:
                #temp and humidity are both good, can turn off that LED
                usinglock = LEDLock.acquire(True)
                if usinglock:
                    stop_blink(LEDStates,4)
                    LEDLock.release()
        
        #Brightness check
        if bright < lowB:
            start_blink(LEDStates,5)
            print("ALERT: Brightness is lower than threshold!")
        elif bright > highB:
            start_blink(LEDStates,5)
            print("ALERT: Brightness is higher than threshold!")
        else:
            usinglock = LEDLock.acquire(True)
            if usinglock:
                stop_blink(LEDStates,5)
                LEDLock.release()
        
        
        try:
            aio.send(aio.temperature_feed.key, temp)
            aio.send(aio.humidity_feed.key, hum)
            aio.send(aio.brightness_feed.key, bright)
            print('\tData Sent to Adafruit.io')
            if numberOfFailedPushes > 0:
                numberOfFailedPushes = 0
                usinglock = LEDLock.acquire(True)
                if usinglock:
                    stop_blink(LEDStates,0)
                    stop_blink(LEDStates,1)
                    LEDLock.release()
            led_on(LEDStates,3)
            time.sleep(0.3) #A blink on LED 3 indicates a successful push to the cloud
            led_off(LEDStates,3)
        except:
            print("Error pushing data to adafruit.io")
            numberOfFailedPushes += 1
            if numberOfFailedPushes >= 5:
                start_blink(LEDStates,0)
                start_blink(LEDStates,1)
        
        wake_time = start_time + SAMPLE_PERIOD
        sleep_time = wake_time - time.time()
        if (sleep_time > 0):
            time.sleep(sleep_time)
    
def updateBrightnessThresholds(aio,lbtFeed,hbtFeed):
    lowB = int(aio.receive(lbtFeed.key).value)
    highB = int(aio.receive(hbtFeed.key).value)
    print("Brightness threshold updated: {} to {}".format(lowB,highB))
    return [lowB,highB]

def updateTemperatureThresholds(aio,lttFeed,httFeed):
    lowT = float(aio.receive(lttFeed.key).value)
    highT = float(aio.receive(httFeed.key).value)
    print("Temperature threshold updated: {}C to {}C".format(lowT,highT))
    return [lowT,highT]

def updateHumidityThresholds(aio,lhtFeed,hhtFeed):
    lowH = float(aio.receive(lhtFeed.key).value)
    highH = float(aio.receive(hhtFeed.key).value)
    print("Humidity threshold updated: {}% to {}%".format(lowH,highH))
    return [lowH,highH]

In [9]:
def client_process(client_id, adakeypath):
    
    with open(adakeypath) as file:
        UNAME = file.readline().rstrip('\n')
        KEY = file.readline().rstrip('\n')
        
    client_id = client_id
    
    global stop
    global SAMPLE_PERIOD 
    SAMPLE_PERIOD = 60*2 #number of seconds to wait between each sample
    global SAMPLES_BEFORE_THRESHOLD_UPDATE
    SAMPLES_BEFORE_THRESHOLD_UPDATE = 60*5  #number of samples to take before updating threshold again

    #custom class see "src/aio_helper_classes.py"
    aio = HabitatMonitorClient(UNAME, KEY, client_id)

    stop = False

    LEDStates = [0, 0, 0, 0, 0, 0] #store states of LEDs. 0 = steady, 1 = blinking

    LEDLock = threading.Lock()

    btns = base.btns_gpio

    t1 = threading.Thread(target=client_sensors, args=(aio,LEDStates,LEDLock))
    t1.start()
    t2 = threading.Thread(target=client_LED, args=(LEDStates,LEDLock)) 
    t2.start()
    t3 = threading.Thread(target=client_button, args=(btns,))
    t3.start()
    t1.join()
    print('t1 joined')
    t2.join()
    print('t2 joined')
    t3.join()
    print('t3 joined')
    print('Client process exited.')

In [10]:
client_id = 'vinit-pynq-board'
adakeypath = "adafruitkey.dontpush"
client_process(client_id, adakeypath)

creating new HabitatMonitor...
HabitatMonitorClientsetup


KeyboardInterrupt: 

# Server Code
## ToDo
Create a server that can monitor the cloud feeds and change thresholds

In [4]:

class serverThresholdParam():
    def __init__(self, ht,lt,hh,lh,hb,lb):
        self.ht = ht
        self.lt = lt
        self.hh = hh
        self.lh = lh
        self.hb = hb
        self.lb = lb

In [5]:
server_leds = {
    0 : base.leds[0],
    1 : base.leds[1],
    2 : base.leds[2],
    3 : base.leds[3],
    4 : base.rgbleds[4],
}

def server_led_blink(blink_time, blink_rate, n):
    '''
    Function to blink the LEDs
    Params:
      blink_time: total time to blink led
      blink_rate: frequency to blink led
      num: the led id to blink
      c: optional argument to specify rbg color. defaults to green
    '''
    global server_leds
    #check for valid args
    t = round(blink_time/blink_rate) #converting the time into counter value
    for i in range(t):
        if 0 <= n and n< 4:
            server_leds[n].toggle()
            time.sleep(1/blink_rate)
        else:
            print("Invalid LED IDs to blink")
            return
    if 0 <= n and n < 4:
        server_leds[n].off()
    if n == 4:
        rgbled.RGBLED(n).write(0)

def server_turn_off_leds(leds):
    '''
    Function to turn off all leds
    Params:
      leds: the leds to turn off
    '''
    for key in leds:
        leds[key].off()

In [6]:
ERROR_RATE = 100#hz
STABLE_RATE = 10 #hz
server_led_blink(2, ERROR_RATE,0)

In [11]:
class HabitatMonitorServer(Client):
    def __init__(self, aio_uname, aio_key, server_id):
        super().__init__(aio_uname, aio_key)
        print('Creating Habitat Mon Server...')
        self.server_id = server_id
        self.client_ids = self.get_client_ids()
        self.high_bright_feed = self.feeds('high-brightness-threshold')
        self.high_hum_feed= self.feeds('high-humidity-threshold')
        self.high_temp_feed = self.feeds('high-temperature-threshold')
        self.low_bright_feed = self.feeds('low-brightness-threshold')
        self.low_hum_feed = self.feeds('low-humidity-threshold')
        self.low_temp_feed = self.feeds('low-temperature-threshold')
        self.temperature_group = self.groups('temperature-measurements')
        self.humidity_group = self.groups('humidity-measurements')
        self.brightness_group = self.groups('brightness-measurements')
        self.client_status_feed = self.feeds('client-status')
        self.high_temp, self.low_temp = None, None
        self.high_hum, self.low_hum = None, None
        self.high_bright, self.low_bright = None, None
        self.HIGH_TEMP_MSG, self.LOW_TEMP_MSG = 'high temp', 'low temp'
        self.HIGH_HUM_MSG, self.LOW_HUM_MGS = 'high hum', 'low hum'
        self.HIGH_BRIGHT_MSG, self.LOW_BRIGHT_MSG = 'high bright', 'low bright'
    def get_client_ids(self):
        client_id_list = []
        id_feed = self.feeds('client-ids')
        cids = self.data(id_feed.key)
        for client_id in cids:
            client_id_list.append(client_id.value)
        return client_id_list
    
    @property
    def high_temp(self):
        return self.__high_temp
    @high_temp.setter
    def high_temp(self, val):
        if val != None:
            self.send(self.high_temp_feed.key, val)
        self.__high_temp = val
    @property
    def high_hum(self):
        return self.__high_hum
    @high_hum.setter
    def high_hum(self, val):
        if val != None:
            self.send(self.high_hum_feed.key, val)
        self.__high_hum = val
    @property
    def high_bright(self):
        return self.__high_bright
    @high_bright.setter
    def high_bright(self, val):
        if val != None:
            self.send(self.high_bright_feed.key, val)
        self.__high_bright = val
    @property
    def low_temp(self):
        return self.__low_temp
    @low_temp.setter
    def low_temp(self, val):
        if val != None:
            self.send(self.low_temp_feed.key, val)
        self.__low_temp = val
    @property
    def low_hum(self):
        return self.__low_hum
    @low_hum.setter
    def low_hum(self, val):
        if val != None:
            self.send(self.low_hum_feed.key, val)
        self.__low_hum = val
    @property
    def low_bright(self):
        return self.__low_bright
    @low_bright.setter
    def low_bright(self, val):
        if val != None:
            self.send(self.low_bright_feed.key, val)
        self.__low_bright = val       

In [12]:
def server_process(uname, key, serverName, serverConfig):
    #create an aio client 
    aio = HabitatMonitorServer(uname, key, serverName)
    #note these lines also push the new value to the could automatically
    aio.high_temp, aio.low_temp = serverConfig.ht, serverConfig.lt 
    aio.high_hum, aio.low_hum = serverConfig.hh, serverConfig.lh
    aio.high_bright, aio.low_bright = serverConfig.hb, serverConfig.lb
    
    btns = base.btns_gpio
    ##Create thread to monitor threshold
    st1 = threading.Thread(target=monitor_thresholds, args=(aio,btns))
    st1.start()
    st1.join()
    
def monitor_thresholds(aio:HabitatMonitorServer, btns):
    monitor_temp_thresholds(aio)
    monitor_hum_thresholds(aio)
    monitor_bright_thresholds(aio)
    
def monitor_temp_thresholds(aio:HabitatMonitorServer):
    for tfeed in aio.temperature_group.feeds:
        print(tfeed.key)
        last_reading = float(aio.receive(tfeed.key).value)
        if last_reading > aio.high_temp:
            print('\tWarning: Temp={} > {}!'.format(last_reading, aio.high_temp))
            server_led_blink(2, ERROR_RATE, 0)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(tfeed.key, aio.HIGH_TEMP_MSG))
        elif last_reading < aio.low_temp:
            print('\tWarning: Temp={} < {}!'.format(last_reading, aio.low_temp))
            server_led_blink(2, ERROR_RATE, 0)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(tfeed.key, aio.LOW_TEMP_MSG))
        else:
            print('\tAll Good: {} < Temp={} < {}'.format(aio.low_temp, last_reading, aio.high_temp))

def monitor_hum_thresholds(aio:HabitatMonitorServer):
    for hfeed in aio.humidity_group.feeds:
        print(hfeed.key)
        last_reading = float(aio.receive(hfeed.key).value)
        if last_reading > aio.high_hum:
            print('\tWarning: Hum={} > {}!'.format(last_reading, aio.high_hum))
            server_led_blink(2, ERROR_RATE, 0)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(hfeed.key, aio.HIGH_HUM_MSG))
        elif last_reading < aio.low_hum:
            print('\tWarning: Hum={} > {}!'.format(last_reading, aio.low_hum))
            server_led_blink(2, ERROR_RATE, 0)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(hfeed.key, aio.LOW_HUM_MSG))
        else:
            print('\tAll good {} < Hum={} < {}'.format(aio.low_hum, last_reading, aio.high_hum))
        
def monitor_bright_thresholds(aio:HabitatMonitorServer):
    for bfeed in aio.brightness_group.feeds:
        print(bfeed.key)
        last_reading = float(aio.receive(bfeed.key).value)
        if last_reading > aio.high_bright:
            print('\tWarning: Bright={} > {}!'.format(last_reading, aio.high_bright))
            server_led_blink(2, ERROR_RATE, 0)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(bfeed.key, aio.HIGH_BRIGHT_MSG))
        elif last_reading < aio.low_bright:
            print('\tWarning: Bright={} < {}!'.format(last_reading, aio.low_bright))
            server_led_blink(2, ERROR_RATE, 0)
            aio.send(aio.client_status_feed.key, '{}, {}'.format(bfeed.key, aio.LOW_BRIGHT_MSG))
        else:
            print('\tAll good {} < Bright={} < {}'.format(aio.low_bright, last_reading, aio.high_bright))

### Main Server function
* If server does not run other client overwrites their own threshold
* If server runs, server overwrites threshold.

In [13]:
with open('adafruitkey.dontpush') as file:
    UNAME = file.readline().rstrip('\n')
    KEY = file.readline().rstrip('\n')

    high_temp   = 100#   = float(input("Enter Higher temperature threshold "))
    low_temp    = 50 #= float(input("Enter Lower temperature threshold "))
    high_hum    = 90 #= float(input("Enter Higher humidity threshold "))
    low_hum     = 25 #= float(input ("Enter lower humidity threshold "))
    high_bright =1000#= float(input("Enter Higher threshold bright light "))
    low_bright  = 800#= float(input("Enter lower threshold bright light "))
    
    serverConfig = serverThresholdParam(high_temp, low_temp, high_hum, low_hum, high_bright, low_bright)
    

    server_process(UNAME, KEY, 'server_1', serverConfig)

Creating Habitat Mon Server...
temperature-measurements.temperature-joeys-pynq-board-1
temperature-measurements.temperature-ryans-pynq-board-2
temperature-measurements.temperature-vinit-pynq-board-3
humidity-measurements.humidity-joeys-pynq-board-1
	All good 25 < Hum=60.0 < 90
humidity-measurements.humidity-ryans-pynq-board-2
	All good 25 < Hum=50.0 < 90
humidity-measurements.humidity-vinit-pynq-board-3
	All good 25 < Hum=42.0 < 90
brightness-measurements.brightness-joeys-pynq-board-1
brightness-measurements.brightness-ryans-pynq-board-2
brightness-measurements.brightness-vinit-pynq-board-3


# main() Function
## ToDo
- Launch client process 
- Optionally launch server process

In [ ]:
def main():
    pass